In [45]:
import numpy as np
import re

In [48]:
def load_A_b_from_m_file(filename):
    with open(filename, 'r') as f:
        content = f.read()

    # Usuń komentarze i zbędne białe znaki
    content = re.sub(r'%.*', '', content)

    # Parsowanie macierzy A
    A_match = re.search(r'A\s*=\s*\[(.*?)\];', content, re.DOTALL)
    if not A_match:
        raise ValueError("Nie znaleziono macierzy A")
    A_rows = A_match.group(1).strip().split('\n')
    A = [list(map(float, row.strip().split())) for row in A_rows]

    # Parsowanie wektora b – obsługuje też ' na końcu
    b_match = re.search(r'b\s*=\s*\[(.*?)\](\'?);', content, re.DOTALL)
    if not b_match:
        raise ValueError("Nie znaleziono wektora b")
    b_data = b_match.group(1).strip().replace('\n', ' ').split()
    b = np.array(list(map(float, b_data)))
    if b_match.group(2) == "'":  # był apostrof, więc wektor pionowy
        b = b.reshape(-1, 1)
    return np.array(A), b

In [43]:
A = np.array([[1,2,5],
            [3,8,1],
            [2,-1,4]])
A

array([[ 1,  2,  5],
       [ 3,  8,  1],
       [ 2, -1,  4]])

In [42]:
n = A.shape[0]
for i in range(0,n):
    for j in range(i+1,n):
        e = A[j][i]/A[i][i]
        for k in range(i,n):
            A[j][k] = A[j][k]-e*A[i][k]
A


array([[  1,   2,   5],
       [  0,   2, -14],
       [  0,   0, -41]])

In [44]:
A = np.array([[1,2,5],
            [3,8,1],
            [2,-1,4]])
A

array([[ 1,  2,  5],
       [ 3,  8,  1],
       [ 2, -1,  4]])

1. Metoda Jacobiego

In [ ]:
A, b = load_A_b_from_m_file('mac_1.m')


In [56]:
x = np.zeros(b.size)
x

array([0., 0., 0., 0., 0., 0., 0.])

In [ ]:
def jacobi(A, b, iter):
    n = b.size
    x = np.zeros(n)
    for iter in range(iter):
        for i in range(n):
            sum = 0
            for j in range(n):
                if(j!=i):
                    sum += A[i][j]*x[j] 
            x[i] = 1/A[i][i]*(b[i]-sum)

    return x

print(jacobi(A,b))

[ -3.33333333   0.          -3.33333333  -6.66666667 -13.33333333
  13.33333333  10.        ]


/tmp/ipykernel_7794/1674003852.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[i] = 1/A[i][i]*(b[i]-sum)
